 ## Writing workflow
 The library accepts any 2d or 3d numpy arrays: (z,y,x) axis order as `stack`, or (y,x) as `plane`. 
 The input array is converted to `uint16` automatically.
 
 1. A writer object is created: it opens a new h5 file and requires info about setups and saving options: 
 number of setup attributes (e.g. channels, angles), compression, subsampling. 
 2. Numpy stacks (3d arrays) can be appended to the h5 file 
 as views by `BdvWriter.append_view(stack, ...)`. 
 Stacks can be normal (whole stack appended at once), or virtual, appended by plane or by substack.
 The time point `time` and attributes such as `channel`, `angle` etc must be specified 
 for each view.
 
 3. An XML file with all meta information is created in the end by `BdvWriter.write_xml(...)`.
  
 4. Writing is finalized by calling `BdvWriter.close()`.
 
**Todo:** 
- examples of reading and cropping views in existing file using `npy2bdv.BdvEditor()` class.
- examples of appending and reading transforms to/from XML file. 

In [1]:
import time
import sys
import os
import numpy as np
import npy2bdv

In [ ]:
# optional: upgrade to latest version
! pip install -U npy2bdv 

In [ ]:
# optional: check installed version
! pip show npy2bdv

In [2]:
def generate_test_image(dim_yx, iz, nz):
    """Gaussian blob spanning the whole range of uint16 type"""
    x = np.linspace(-3, 3, dim_yx[1])
    y = np.linspace(-3, 3, dim_yx[0])
    sigma = 1.0 - abs(iz - nz/2) / nz
    x, y = np.meshgrid(x, y)
    return (65535 * np.exp(- ((x ** 2) + (y ** 2)) / (2 * sigma**2) )).astype('uint16')

def generate_test_stack(nz, ny, nx):
    stack = np.empty((nz, ny, nx))
    for z in range(nz):
        stack[z, :, :] = generate_test_image((ny, nx), z, nz)
    return stack.astype('uint16')
    
examples_dir = "./example_files/"
if not os.path.exists(examples_dir):
    os.mkdir(examples_dir)

## 1. Basic stack writing

### Stacks (views) are written as raw data.  
No compression, no downsampling. This mode ensures highest speed, but may be sub-optimal for post-processing and data storage that requires compression and downsampling. See Example 5 for more options.

In [54]:
%%time
nz, ny, nx = 50, 1024, 2048
stack = generate_test_stack(nz, ny, nx)
fname = examples_dir + "ex1_t2_ch2_illum2_angle2_raw.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, nilluminations=2, nangles=2)

bdv_writer.set_attribute_labels('channel', ('488', '561'))
bdv_writer.set_attribute_labels('illumination', ('L', 'R'))
bdv_writer.set_attribute_labels('angle', ('90', '180'))

for t in range(2):
    for i_ch in range(2):
        for i_illum in range(2):
            for i_angle in range(2):
                bdv_writer.append_view(stack, time=t, channel=i_ch, illumination=i_illum, angle=i_angle)

bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset in {fname}")

dataset in ./example_files/ex1_t2_ch2_illum2_angle2_raw.h5
Wall time: 9.03 s


## 2. Writing speed test

Speed test for raw data writing, 20 time points and 2 channels. File size is 17 GB.

In [24]:
nt, nc, nz, ny, nx = 20, 2, 50, 2048, 2048
stack = generate_test_stack(nz, ny, nx)

fname = examples_dir + "ex2_t20_chan2.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=nc)

start_time = time.time()
print('clock on:',int(start_time))

for ichannel in range(nc):
    for itime in range(nt):
        start_time = time.time()
        bdv_writer.append_view(stack, time=itime, channel=ichannel)

bdv_writer.write_xml()
bdv_writer.close()

stop_time = time.time()
print('clock off:', int(stop_time))
time_per_stack = (stop_time - start_time) / (nt*nc)

print(f"H5 mean writing time per stack: {time_per_stack:1.3f} sec.")
print(f"H5 mean writing speed: {int(sys.getsizeof(stack) / time_per_stack / 1e6)} MB/s")
print(f"dataset in {fname}")

clock on: 1721819333
clock off: 1721819398
H5 mean writing time per stack: 0.046 sec.
H5 mean writing speed: 9042 MB/s
dataset in ./example_files/ex2_t20_chan2.h5


**Warning:** This speed should not be trusted: time variables `start_time` and `stop_time` change during cell execution, see below:

In [27]:
print('clock on:',int(start_time))
print('clock off:', int(stop_time))
stop_time - start_time

clock on: 1721819396
clock off: 1721819398


1.855459213256836

## 3. Writing with affine transformations defined in XML file

Affine transformations define translation, rotation, scaling, and shear.

In this example we write 1 time point and 1 channel with 10-px shear transformation along X axis. With non-isotropic voxel size calibration.
      
The affine transformation defined in XML file is automatically applied every time you open the dataset in BigDataViewer/BigStitcher.

In [49]:
shear_x_px = 10
affine_matrix = np.array(((1.0, 0.0, -shear_x_px, 0.0),
                          (0.0, 1.0, 0.0, 0.0),
                          (0.0, 0.0, 1.0, 0.0)))

fname = examples_dir + "ex3_t1_ch1_shear.h5"
bdv_writer = npy2bdv.BdvWriter(fname)
bdv_writer.append_view(stack, time=0, channel=0,
                       m_affine=affine_matrix,
                       name_affine="shearing transformation",
                       calibration=(1, 1, 1))
bdv_writer.write_xml()
bdv_writer.close()
print(f"sheared dataset in {fname}")

sheared dataset in ./example_files/ex3_t1_ch1_shear.h5


## 3.1 Writing multiple tiles on a grid

Here, affine transformation matrix defines translation of individual tiles in the global coordinate system.
Tile position is the position of its lower left corner. The last (4-th) column of the affine matrix defines the translation terms (x,y,z).

Optionally, make some tiles missing.

In [50]:
DEBUG_MODE = False
MISSING_TILES = False

n_tiles_x = 4
n_tiles_y = 2

tile_w, tile_h = stack.shape[2] // n_tiles_x, stack.shape[1] // n_tiles_y

tile_position_yx = ((0,0), (0, tile_w), (tile_h, 0), (tile_h, tile_w))

unit_matrix = np.array(((1.0, 0.0, 0.0, 0.0), # change the 4. value for x_translation (px)
                        (0.0, 1.0, 0.0, 0.0), # change the 4. value for y_translation (px)
                        (0.0, 0.0, 1.0, 0.0)))# change the 4. value for z_translation (px)

fname = examples_dir + "ex3p1_tiles.h5"
bdv_writer = npy2bdv.BdvWriter(fname, ntiles=n_tiles_x*n_tiles_y)
for j in range(n_tiles_y):
    for i in range(n_tiles_x):
        tile_stack = stack[:, j*tile_h:(j+1)*tile_h, i*tile_w:(i+1)*tile_w]
        tile_index = i + j*n_tiles_x
        affine_matrix = unit_matrix
        affine_matrix[0,3] = i * tile_w # x-translation
        affine_matrix[1,3] = j * tile_h # y-translation
        if DEBUG_MODE:
            print(f"Tile index: {tile_index}")
            print(f"Transform matrix: \n {affine_matrix}")
        if MISSING_TILES and tile_index in (1,2): # make two tiles missing 
            pass
        else:
            bdv_writer.append_view(tile_stack, time=0, 
                                   tile=tile_index, 
                                   m_affine=affine_matrix, 
                                   name_affine=f"tile {tile_index} translation")
bdv_writer.write_xml()
bdv_writer.close()
print(f"\n File with multiple tiles: {fname}")


 File with multiple tiles: ./example_files/ex3p1_tiles.h5


## 4. Writing with experiment metadata
Dataset with 1 time point and 1 channel with voxel size, exposure, camera and microscope properties stored in XML.

In [51]:
fname = examples_dir + "ex4_cam_props.h5"
bdv_writer = npy2bdv.BdvWriter(fname)
bdv_writer.append_view(stack, time=0, 
                       voxel_size_xyz=(1, 1, 5), voxel_units='um',
                       exposure_time=10, exposure_units='ms')
bdv_writer.write_xml(camera_name="Hamamatsu OrcaFlash100",
                          microscope_name='Superscope',
                          user_name='nvladimus')
bdv_writer.close()
print(f"dataset is in {fname}")

dataset is in ./example_files/ex4_cam_props.h5


## 5 Writing with downsampling and compression
### 5.1 Stacks are written with downsampling and compression on-the-fly. 
This mode is usually too slow for image writing during experiment, but offers user convenience for offline processing.

In [52]:
%%time
fname = examples_dir + "ex5_t2_ch2_illum2_angle2_realtime_downsamp.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, nilluminations=2, nangles=2, 
                               subsamp=((1, 4, 4),    (2, 8, 8),    (4, 16, 16)),
                               blockdim=((8, 32, 32), (16, 32, 32), (32, 32, 32)),
                               compression='gzip')

bdv_writer.set_attribute_labels('channel', ('488', '561'))
bdv_writer.set_attribute_labels('illumination', ('L', 'R'))
bdv_writer.set_attribute_labels('angle', ('90', '180'))

for t in range(2):
    for i_ch in range(2):
        for i_illum in range(2):
            for i_angle in range(2):
                bdv_writer.append_view(stack, time=t, channel=i_ch, illumination=i_illum, angle=i_angle)

bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset in {fname}")

dataset in ./example_files/ex5_t2_ch2_illum2_angle2_realtime_downsamp.h5
Wall time: 24.9 s


### 5.2 Stacks are written as raw data followed by lazy downsampling and compression. 
This mode combines highest speed during acqusition (raw data writing) with delayed (lazy) downsampling and compression.

In [53]:
fname = examples_dir + "ex5_t2_ch2_illum2_angle2_lazy_downsamp.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, nilluminations=2, nangles=2)

bdv_writer.set_attribute_labels('channel', ('488', '561'))
bdv_writer.set_attribute_labels('illumination', ('L', 'R'))
bdv_writer.set_attribute_labels('angle', ('90', '180'))

start_time = time.time()
for t in range(2):
    for i_ch in range(2):
        for i_illum in range(2):
            for i_angle in range(2):
                bdv_writer.append_view(stack, time=t, channel=i_ch, illumination=i_illum, angle=i_angle)

print(f"Raw data writing time, total: {(time.time() - start_time):2.2f} sec.")

start_time = time.time()
bdv_writer.create_pyramids(subsamp=((1, 4, 4),    (2, 8, 8),    (4, 16, 16)), 
                           blockdim=((8, 32, 32), (16, 32, 32), (32, 32, 32)),
                          compression='gzip')    
print(f"Downsampling & compression & pyramid writing time, total: {(time.time() - start_time):2.2f} sec.")

bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset in {fname}")

views:   0%|                                                                                     | 0/8 [00:00<?, ?it/s]

Raw data writing time, total: 4.94 sec.



time points: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:24<00:00, 12.36s/it]

Downsampling & compression & pyramid writing time, total: 24.75 sec.
dataset in ./example_files/ex5_t2_ch2_illum2_angle2_lazy_downsamp.h5


## 6. Writing virtual stacks that are too big to fit RAM
### 6.1 Writing by plane
Dataset has 1 time point, 2 channels, and large virtual stack, written by plane.

In [47]:
%%time
nc, nz, ny, nx = 2, 400, 2048, 2048
test_image = generate_test_image((ny, nx), nz/2, nz)
fname = examples_dir + "ex6_t1_ch2_virtual_by_plane.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, blockdim=((1, 256, 256),))

start_time = time.time()
for i_ch in range(nc):
    bdv_writer.append_view(stack=None, virtual_stack_dim=(nz,ny,nx), time=0, channel=i_ch)
    for z in range(nz):
        bdv_writer.append_plane(plane=test_image, z=z, time=0, channel=i_ch)

bdv_writer.write_xml()
bdv_writer.close()

time_total = time.time() - start_time
ave_time_per_plane = time_total/(nc*nz)
print(f"H5 mean writing speed: {int(sys.getsizeof(test_image)*nc*nz / time_total / 1e6)} MB/s")
print(f"H5 mean writing time per plane: {ave_time_per_plane:1.3f} sec.")
print(f"virtual stack in {fname}")

H5 mean writing speed: 216 MB/s
H5 mean writing time per plane: 0.039 sec.
virtual stack in ./example_files/ex6_t1_ch2_virtual_by_plane.h5
Wall time: 31.1 s


### 6.2 Virtual stack, writing by substack

In [51]:
%%time
nc, nz, ny, nx, n_substacks = 2, 400, 2048, 2048, 4
stack = generate_test_stack(nz, ny, nx)
fname = examples_dir + "ex6_t1_ch2_virtual_by_substack.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, blockdim=((32, 32, 32),))

print('clock started')
start_time = time.time()
#initialize virtual stacks
for i_ch in range(nc):
    bdv_writer.append_view(stack=None, virtual_stack_dim=(nz,ny,nx), time=0, channel=i_ch)

# populate virtual stacks 
for i_ch in range(nc):
    for isub in range(n_substacks):
        zslice = slice(isub*(nz//n_substacks), (isub+1)*(nz//n_substacks))
        bdv_writer.append_substack(substack=stack[zslice, :, :],
                                   z_start=zslice.start,
                                   channel=i_ch)

bdv_writer.write_xml()
bdv_writer.close()

time_total = time.time() - start_time
ave_time_per_plane = time_total/(nc*nz)

print(f"H5 mean writing speed: {int(sys.getsizeof(stack) * nc / time_total / 1e6)} MB/s")
print(f"H5 mean writing time per plane: {ave_time_per_plane:1.3f} sec.")
print(f"virtual stack in {fname}")

clock started
H5 mean writing speed: 325 MB/s
H5 mean writing time per plane: 0.026 sec.
virtual stack in ./example_files/ex6_t1_ch2_virtual_by_substack.h5
Wall time: 2min 3s


### 6.a Virtual stacks by plane, with downsampling

Subsampling allows faster viewing in BDV, because only the necessary level of details is loaded into RAM, instead of full resolution image.

When writing by plane, the subsampling level Z must be 1, so there is no way to subsample in Z on the fly. For example, `subsamp=((1, 1, 1), (1, 4, 4), (1, 8, 8))`.

The newly generated file is slightly bigger (by ca. 8%), but the processing time increases 5x.

Compression is supported.

In [53]:
%%time
nc, nz, ny, nx = 2, 400, 2048, 2048
test_image = generate_test_image((ny, nx), nz/2, nz)
fname = examples_dir + "ex6a_t1_ch1_virtual_by_plane_subsamp.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2,
                               blockdim=((1, 128, 128),),
                               subsamp=((1, 1, 1), (1, 4, 4), (1, 8, 8)))
print('clock started')
start_time = time.time()
for i_ch in range(nc):
    bdv_writer.append_view(stack=None, virtual_stack_dim=(nz,ny,nx), time=0, channel=i_ch)
    for z in range(nz):
        bdv_writer.append_plane(plane=test_image, z=z, time=0, channel=i_ch)

bdv_writer.write_xml()
bdv_writer.close()

time_total = time.time() - start_time
ave_time_per_plane = time_total/(nc*nz)

print(f"H5 mean writing speed: {int(sys.getsizeof(test_image) * nc * nz / time_total / 1e6)} MB/s")
print(f"H5 mean writing time per plane: {ave_time_per_plane:1.3f} sec.")
print(f"virtual stack in {fname}")

INFO: blockdim levels (1) < subsamp levels (3): First-level block size (1, 128, 128) will be used for all levels
clock started
H5 mean writing speed: 161 MB/s
H5 mean writing time per plane: 0.052 sec.
virtual stack in ./example_files/ex6a_t1_ch1_virtual_by_plane_subsamp.h5
Wall time: 41.8 s


## 7. Missing views, normal stack
Missing views are detected automatically and marked as such in XML file.

In [57]:
nz, ny, nx = 50, 1024, 2048
stack = generate_test_stack(nz, ny, nx)
fname = examples_dir + "ex7_missing_views.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2)
bdv_writer.append_view(stack, time=0, channel=0)
bdv_writer.append_view(stack, time=1, channel=1)
bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset with missing views in {fname}")

dataset with missing views in ./example_files/ex7_missing_views.h5


## 8. Missing views, virtual stack by plane

In [58]:
nz, ny, nx = 50, 1024, 2048
stack = generate_test_stack(nz, ny, nx)
fname = examples_dir + "ex8_virtual_stack_missing_views.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2)
bdv_writer.append_view(stack=None, virtual_stack_dim=(nz, ny, nx), time=0, channel=0)
bdv_writer.append_view(stack=None, virtual_stack_dim=(nz, ny, nx), time=1, channel=1)

for z in range(nz):
    bdv_writer.append_plane(plane=test_image, z=z, time=0, channel=0)
    bdv_writer.append_plane(plane=test_image, z=z, time=1, channel=1)

bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset with missing views in {fname}")

dataset with missing views in ./example_files/ex8_virtual_stack_missing_views.h5


## Clean up the data files generated

In [17]:
import shutil
if os.path.exists(examples_dir):
    shutil.rmtree(examples_dir)